In [1]:
# from kubernetes_asyncio import client, config
import kubernetes_asyncio as kube
import asyncio

In [2]:

async def list_pods_once():
	
	await kube.config.load_kube_config()

	api = kube.client.CoreV1Api()
	
	pod_list_response =await api.list_namespaced_pod('cvlab')
	
	print(' '.join(f'{pod.metadata.name}' for pod in pod_list_response.items))
	
	return pod_list_response
	
# asyncio.run(main())
pod_list = await list_pods_once()


# Configs can be set in Configuration class directly or using helper
# utility. If no argument provided, the config will be loaded from
# default location.


egundogdugarment3 egundogdugarment4 egundogdugarment5 lis-054x-0124 lis-08x0-glow-simple-decay lis-psp-ens-nyu lis-test-home-var-with-security-ctx lis-test-sec-context nakka-fcn8 pytorch-arch-transfer vidit-maskrcnn-seg-res50 weliu-fdst weliu-worldbase


In [3]:
from datetime import date, datetime, timezone

now = datetime.now(timezone.utc)
now - pod_obj.metadata.creation_timestamp

NameError: name 'pod_obj' is not defined

In [4]:
import json
from functools import partial

def json_serialize_unknown(obj):
	if isinstance(obj, (datetime, date)):
		return obj.isoformat()
	
	raise TypeError(f'Object not serializable class={type(obj)} obj={obj}')
	
json_serialize = partial(json.dumps, default=json_serialize_unknown)



In [5]:
import logging, sys
log = logging.getLogger(__name__)
# ch = logging.StreamHandler()
# ch.setLevel(logging.INFO)
log.setLevel(logging.INFO)
log.addHandler(logging.StreamHandler(sys.stdout))
log.addHandler(logging.handlers.RotatingFileHandler('watchdog.log', maxBytes=1024*1024, backupCount=7))


In [6]:
log.info('bla')

bla


INFO:__main__:bla


In [5]:
from dataclasses import dataclass
from datetime import date, datetime, timezone


@dataclass
class PodInfo:
	
	name: str
	user: str
	status: str
	date_created: datetime
	date_started: datetime
	num_gpu: int # number of GPUs used
		
	user_priority: int # priority specified by the user, the higher the more important, 0 is default
	user_queue_position: int # position in owner's queue: is it the 1st, 2nd ... most important job
	
	def __init__(self, pod_obj):	
		labels = pod_obj.metadata or {} # if null then use empty dict

		self.name = pod_obj.metadata.name
		self.user = labels.get('user', None)
		self.status = pod_obj.status.phase,

		self.num_gpu = self.extract_num_gpu(pod_obj)
		
		self.user_priority = self.extract_priority(pod_obj)
		self.user_queue_position = 0
		
		self.date_created = pod_obj.metadata.creation_timestamp,
		self.date_started = self.extract_started_at(pod_obj)
		
	@staticmethod
	def extract_priority(pod_obj):
		labels = pod_obj.metadata or {} # if null then use empty dict
		
		priority_label = labels.get('priority', 0)
		
		try:
			return int(priority_label)
		except ValueError:
			log.info(f'Non-numeric value for labels|priority: {priority} in pod {pod_obj.metadata.name}')
			return 0 # default
	
	@staticmethod
	def extract_num_gpu(pod_obj):
		num_gpu = 0
		for container in pod_obj.spec.containers:
			limits = container.resources.limits
			if limits is not None:
				num_gpu_in_container = limits.get('nvidia.com/gpu', 0)
				try:
					num_gpu += int(num_gpu_in_container)
				except ValueError:
					log.warning(f'Unexpected value for limits|nvidia.com/gpu: {num_gpu_in_container} in pod {pod_obj.metadata.name}')

		return num_gpu
					
	@staticmethod
	def extract_started_at(pod_obj):
		started_at = None
		for status in pod_obj.status.container_statuses:
			if status.state.running is not None:
				started_at = status.state.running.started_at # TODO get earlier date
		return started_at

	def __repr__(self):
		return f'pod({self.name}, user={self.user} with priority={self.user_priority}, {self.status}, {self.num_gpu} GPU)'
	
	

In [7]:
api = kube.client.CoreV1Api()

In [ ]:

import kubernetes_asyncio as kube
import traceback


class KubernetesPodListSupervisor:
	def __init__(self):
		self.pod_data_from_api = {}
		self.pod_info_processed = {}
		
		self.pod_info_list = []
	
	
	async def listen(self):
		
		await kube.config.load_kube_config()
		
		api = kube.client.CoreV1Api()

		w = kube.watch.Watch()

		async for event in w.stream(api.list_namespaced_pod, namespace='cvlab'):
			self.process_event(event)
		
	
	def process_event(self, event):

		try:
			pod_obj = event['object']
			ev_type = event['type']
			pod_name = pod_obj.metadata.name

			log.debug(f"Event: {ev_type} {pod_name}")

			if ev_type == 'MODIFIED' or ev_type == 'ADDED':
				self.pod_data_from_api[pod_name] = pod_obj
				self.pod_info_processed[pod_name] = PodInfo(pod_obj)

			elif ev_type == 'DELETED':
				del self.pod_data_from_api[pod_name]
				del self.pod_info_processed[pod_name]

			else:
				log.error(f'Unusual event type from kubectl: {ev_type}')

			self.pod_info_list = list(self.pod_info_processed.values())
		
		except Exception as e:
			log.exception(f'PodListSupervisor: error while processing event\n {event}')
	
			
			
async def list_pods_online():
	
	await kube.config.load_kube_config()

	api = kube.client.CoreV1Api()
	
	w = kube.watch.Watch()
# 	events = []
	async for event in w.stream(api.list_namespaced_pod, namespace='cvlab'):
# 		events.append(event)
		print(f"Event: {event['type']} {event['object'].metadata.name}")

	
# asyncio.run(main())
await list_pods_online()


In [92]:
import traceback


def get_default_nullable_dict(d, key, default):
	return d.get(key, default) if d is not None else default

def pod_info_extract(pod_obj):
	
	
	info = {
		'name': pod_obj.metadata.name,
		'user': get_default_nullable_dict(pod_obj.metadata.labels, 'user', None),	
		'status': pod_obj.status.phase,
		'date created': pod_obj.metadata.creation_timestamp,
	}
	
	priority = get_default_nullable_dict(pod_obj.metadata.labels, 'priority', 0)
	try:
		priority = int(priority)
	except ValueError:
		print(f'Non-numeric value for labels|priority: {priority} in pod {pod_obj.metadata.name}')
		priority = 0
	
	info['priority'] = 0
			
	num_gpu = 0
	for container in pod_obj.spec.containers:
		limits = container.resources.limits
		if limits is not None:
			num_gpu_in_container = limits.get('nvidia.com/gpu', 0)
			try:
				num_gpu += int(num_gpu_in_container)
			except ValueError:
				print(f'Unexpected value for limits|nvidia.com/gpu: {num_gpu_in_container} in pod {pod_obj.metadata.name}')
			
	info['num_gpu'] = num_gpu
	
	started_at = None
	for status in pod_obj.status.container_statuses:
		if status.state.running is not None:
			started_at = status.state.running.started_at # TODO get earlier date
	info['date started'] = started_at if started_at is not None else info['date created']
	
	return info
		
	
def pod_description(pod_obj):
	pod_data = pod_info_extract(pod_obj)
	
	print('\n'.join(f'{k}:	{v}' for k, v in pod_data.items()))
# 	print(json_serialize(pod_data))



pod_obj = pod_list.items[0]
pod_description(pod_obj)

for pod_obj in pod_list.items:
	try:
		pod_description(pod_obj)
	except Exception as e:
		print('Error', e)
		traceback.print_exc()
		print('In pod\n', pod_obj)
		

# pod_obj

name:	egundogdugarment3
user:	guendogd
status:	Running
date created:	2019-08-10 09:30:13+00:00
priority:	0
num_gpu:	1
date started:	2019-08-10 09:32:34+00:00
name:	egundogdugarment3
user:	guendogd
status:	Running
date created:	2019-08-10 09:30:13+00:00
priority:	0
num_gpu:	1
date started:	2019-08-10 09:32:34+00:00
name:	egundogdugarment4
user:	guendogd
status:	Running
date created:	2019-08-10 09:31:32+00:00
priority:	0
num_gpu:	1
date started:	2019-08-10 09:31:35+00:00
name:	egundogdugarment5
user:	guendogd
status:	Running
date created:	2019-08-11 10:48:33+00:00
priority:	0
num_gpu:	1
date started:	2019-08-11 10:50:30+00:00
name:	lis-054x-0124
user:	lis
status:	Succeeded
date created:	2019-06-24 12:48:45+00:00
priority:	0
num_gpu:	1
date started:	2019-06-24 12:48:45+00:00
name:	lis-08x0-glow-simple-decay
user:	lis
status:	Succeeded
date created:	2019-07-22 12:46:57+00:00
priority:	0
num_gpu:	1
date started:	2019-07-22 12:46:57+00:00
name:	lis-psp-ens-nyu
user:	lis
status:	Succeeded
dat

In [107]:

from operator import itemgetter
from copy import copy
import sys

def sorting_key_user_internal(pod_info):
	"""
	Sorted by the following
	"""
	
	# CPU job is free, always before GPU job
	key_cpu_only = 0 if pod_info['num_gpu'] == 0 else 1

	# User priority, the higher the better, thus the minus
	user_priority = -pod_info['priority']
	
	# Date: newer is better
	pod_date = pod_info['date started']
	
	return (key_cpu_only, user_priority, pod_date, pod_info['name'])


def pods_calc_user_queue(pods_of_user):
	pods_of_user.sort(key=sorting_key_user_internal)
	
	gpu_accumulation = 0
	
	for pod in pods_of_user:
		gpu_accumulation += pod['num_gpu']
		pod['user_score'] = gpu_accumulation

	return pods_of_user

def sorting_key_all_pods(pod_info):
	"""
	Sorted by the following
	"""
	
	# CPU job is free, always before GPU job
	key_cpu_only = 0 if pod_info['num_gpu'] == 0 else 1
	
	# Anonymous always after the ones of a known user
	key_anonymous = 0 if pod_info['user'] is not None else 1
	
	# User priority, the higher the better, thus the minus
	user_score = pod_info['user_score'] if pod_info['user'] is not None else sys.maxsize
	
	# Date: newer is better
	pod_date = pod_info['date started']
	
	return (key_cpu_only, key_anonymous, user_score, pod_date, pod_info['name'])

def hierarchy_calculate(pod_infos):

	pods_by_user = {}
	
	for pod in pod_infos:
		if pod['status'] == 'Running':
			pods_by_user.setdefault(pod['user'], []).append(copy(pod))
	
	pods_all = []
	for user, pods in pods_by_user.items():
		if user is not None:
			pods = pods_calc_user_queue(pods)
		else:
			for p in pods:
				p['user_score'] = sys.maxsize
		
		pods_all += pods
		
	pods_all.sort(key=sorting_key_all_pods)
		
# 	print(pods_all)

	gpu_accumulation = 0

	for p in pods_all:
		gpu_accumulation += p['num_gpu']
		print('{gpu_accumulation} {name}	{user}	{user_score}	[{num_gpu} gpus]'.format(**p))
	
	
	print('\n'.join(
		'{name} {user} {user_score} [{num_gpu} gpus]'.format(**p)
		for p in pods_all
	))
	
	
# 	print('\n'.join(sorted(str(user) + ': ' + ' '.join(
# 		p['name'] for p in pl) for user, pl in pods_by_user.items()
# 	)))
	
pod_info_list = [pod_info_extract(p) for p in pod_list.items]
	
hierarchy_calculate(pod_info_list)
	
	
	

egundogdugarment4 guendogd 1 [1 gpus]
weliu-worldbase weliu 1 [1 gpus]
vidit-maskrcnn-seg-res50 vidit 1 [1 gpus]
egundogdugarment3 guendogd 2 [1 gpus]
pytorch-arch-transfer sguo 2 [2 gpus]
weliu-test weliu 2 [1 gpus]
egundogdugarment5 guendogd 3 [1 gpus]
nakka-fcn8 None 9223372036854775807 [2 gpus]


In [ ]:
class KubernetesPodSupervisor:
	def __init__(self):
		pass
	
	
	def listen(self):
		




async def list_pods_online():
	
	await kube.config.load_kube_config()

	api = kube.client.CoreV1Api()
	
	w = kube.watch.Watch()
# 	events = []
	async for event in w.stream(api.list_namespaced_pod, namespace='cvlab'):
# 		events.append(event)
		print(f"Event: {event['type']} {event['object'].metadata.name}")

	
# asyncio.run(main())
await list_pods_online()


Event: ADDED lis-054x-0124
Event: ADDED egundogdugarment5
Event: ADDED weliu-test
Event: ADDED egundogdugarment3
Event: ADDED weliu-worldbase
Event: ADDED pytorch-arch-transfer
Event: ADDED egundogdugarment4
Event: ADDED lis-psp-ens-nyu
Event: ADDED lis-test-sec-context
Event: ADDED nakka-fcn8
Event: ADDED lis-08x0-glow-simple-decay
Event: ADDED vidit-maskrcnn-seg-res50
Event: ADDED lis-test-home-var-with-security-ctx
Event: ADDED lis-wdog-events
Event: MODIFIED lis-wdog-events
Event: MODIFIED lis-wdog-events
Event: MODIFIED lis-wdog-events
Event: MODIFIED lis-wdog-events
Event: MODIFIED lis-wdog-events
Event: DELETED lis-wdog-events


ERROR:asyncio:SSL error in data received
protocol: <asyncio.sslproto.SSLProtocol object at 0x7f7d972a0c18>
transport: <_SelectorSocketTransport fd=59 read=polling write=<idle, bufsize=0>>
Traceback (most recent call last):
  File "/home/adynathos/Libs/conda/envs/main37/lib/python3.7/asyncio/sslproto.py", line 526, in data_received
    ssldata, appdata = self._sslpipe.feed_ssldata(data)
  File "/home/adynathos/Libs/conda/envs/main37/lib/python3.7/asyncio/sslproto.py", line 207, in feed_ssldata
    self._sslobj.unwrap()
  File "/home/adynathos/Libs/conda/envs/main37/lib/python3.7/ssl.py", line 767, in unwrap
    return self._sslobj.shutdown()
ssl.SSLError: [SSL: KRB5_S_INIT] application data after close notify (_ssl.c:2609)


In [ ]:
# List pods

def list_pods():
	v1 = client.CoreV1Api()
	
	print("Listing pods with their IPs:")
	
	#ret = v1.list_pod_for_all_namespaces(watch=False)
	ret = v1.list_namespaced_pod('cvlab')
	
	pods = list(ret.items)
	
	for pod in pods:
		print(f'{pod.metadata.name}	{pod.metadata.namespace}	{pod.status.pod_ip}')
	
	return pods

pods = list_pods()


In [ ]:
pods[0].metadata.labels

In [ ]:
for pod in pods:
	print(f'{pod.metadata.name}	{pod.metadata.labels}')


In [ ]:
#v1.list_namespaced_replication_controller('cvlab')
#v1.list_namespaced_service('cvlab')
v1.list_na

In [ ]:


def watch_pods():
	v1 = client.CoreV1Api()
	count = 100

	w = watch.Watch()
	events = []
	for event in w.stream(v1.list_namespaced_pod, namespace='cvlab'):
		events.append(event)
		print(f"Event: {event['type']} {event['object'].metadata.name}")
		count -= 1
		if count <= 0:
			w.stop()
			

	print("Ended.")
	
watch_pods()
			  
			  


In [ ]:
#configuration = kubernetes.client.Configuration()

api_instance = client.BatchV1Api()

In [ ]:
api_instance.list_namespaced_job('cvlab')